In [1]:
from dotenv import dotenv_values
import requests

from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel
from helpers.writer_citer_model import WriterCiterModel


"""
============================================
Run through a single trial of the experiment
============================================
"""

"""
Set up
"""

# Number of links to consider
K = 5

# Declare trial number
trial_number = 5

"""
Configuration
"""

sfblic_link = "https://www.sfbli.com/"

# Load environment variables
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
storage_manager = StorageManager("../data/")    
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
corpus_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
improver_model = ImproverModel(api_key=config['OPEN_AI_API_KEY'])
writer_citer = WriterCiterModel(api_key=config['OPEN_AI_API_KEY'])


# Load the clean corpus text
with open("../data/clean_content.txt") as f:
    corpus_text = f.read()


# Select a query to use for the current trial
query = "What is life insurance?"

In [2]:
"""
Submit the query to Google and store the results
"""

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=10)
    
# Store the response
storage_manager.save_to_folder("InitialGoogleResponses", search_results, f"{trial_number}.json")

Data saved to ../data/InitialGoogleResponses/5.json


In [3]:
"""
View links
"""

top_k_links = [item['link'] for item in search_results['items']][:K]
top_k_links

['https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work',
 'https://www.benefits.va.gov/insurance/',
 'https://www.va.gov/life-insurance/options-eligibility/valife/',
 'https://content.naic.org/insurance-topics/life-insurance',
 'https://www.va.gov/life-insurance/']

In [4]:
"""
Check to see if SFBLIC is in the top K links. 
If not, add it
"""

if sfblic_link in top_k_links:
    updated_links = top_k_links
else:
    top_k_links[K-1] = sfblic_link
    updated_links = top_k_links

updated_links

['https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work',
 'https://www.benefits.va.gov/insurance/',
 'https://www.va.gov/life-insurance/options-eligibility/valife/',
 'https://content.naic.org/insurance-topics/life-insurance',
 'https://www.sfbli.com/']

In [5]:
storage_manager.save_to_folder("UpdatedGoogleResponses", updated_links, f"{trial_number}.json")

Data saved to ../data/UpdatedGoogleResponses/5.json


In [6]:
"""
Get Raw Text from the HTML associated with the links from Google
"""

# Retrieve the stored response
updated_links = storage_manager.load_from_folder("UpdatedGoogleResponses", f"{trial_number}.json")

raw_text_from_html = []

# For each item of raw text that was stored ...
for link in updated_links:    

    # Fet the HTML associated with the link
    response = requests.get(link)

    # Get the raw text associated with the HTML
    parsed_text = HTMLParser.parse_html(response.content)

    # Store the raw text from the HTML
    raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})
    
storage_manager.save_to_folder("RawTextFromHTML", raw_text_from_html, f"{trial_number}.json")

Data loaded from ../data/UpdatedGoogleResponses/5.json
Data saved to ../data/RawTextFromHTML/5.json


In [7]:
"""
Get the refined text from the raw text
"""

raw_text_from_html = storage_manager.load_from_folder("RawTextFromHTML", f"{trial_number}.json")

website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")

storage_manager.save_to_folder("WebsiteRefinerModelResponses", website_refiner_responses, f"{trial_number}.json")

Data loaded from ../data/RawTextFromHTML/5.json
Data saved to ../data/WebsiteRefinerModelResponses/5.json


In [8]:
for item in website_refiner_responses:
    print(item['link'])
    print(item['choices'][0]['message']['content'])
    print()

https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work
Life insurance is a contract between an insurance company and policyholder. In exchange for a premium, the life insurance company agrees to pay a sum of money to one or more named beneficiaries upon the death of the policyholder. The purpose of life insurance is to help provide financial security to your loved ones upon your death. Some life policies also offer living benefits, allowing payment of a part of the policy's death benefit while you're still alive if diagnosed with a covered chronic, critical, or terminal illness.

https://www.benefits.va.gov/insurance/
Life insurance is a financial product that provides coverage in the event of death, offering beneficiaries a payout. There are various types of life insurance available:

1. **Veterans Affairs Life Insurance (VALife)** - A guaranteed acceptance whole life insurance program for service-connected Veterans aged 80 and under, providing coverage of

In [9]:
"""
Filter the corpus to information that may be relevant to the query
"""

response_from_corpus_refiner = corpus_refiner.refine_text(raw_text=corpus_text, query=query)
storage_manager.save_to_folder("CorpusRefinerModelResponses", response_from_corpus_refiner, f"{trial_number}.json")

Data saved to ../data/CorpusRefinerModelResponses/5.json


In [10]:
print(response_from_corpus_refiner['choices'][0]['message']['content'])

Life insurance is a financial product that provides coverage to policyholders in exchange for premiums, designed to protect families and provide financial security. Southern Farm Bureau Life Insurance Company offers various life insurance products, including:

1. **Permanent Life Insurance**: This includes whole life, single premium, modified premium, and adjustable premium options, providing lifelong coverage with different payment structures.
2. **Term Life Insurance**: This type of insurance is available in 10, 20, and 30-year intervals, providing cost-effective, time-bound coverage.

The company aims to be the life insurance provider of choice, focusing on competitive offerings, superior customer service, and ethical practices. Its mission statements emphasize a commitment to protecting families and supporting financial security across generations.


In [11]:
"""
Improve the text from the SFBLIC site
"""

# Get the refined text from the website
website_refiner_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")
sfblic_response = [response for response in website_refiner_responses if response['link'] == 'https://www.sfbli.com/'][0]
refined_text_from_website = sfblic_response['choices'][0]['message']['content']

# Get the refined text from the corpus
response_from_corpus_refiner = storage_manager.load_from_folder("CorpusRefinerModelResponses", f"{trial_number}.json")
refined_text_from_corpus = response_from_corpus_refiner['choices'][0]['message']['content']


# Generate improved text
improved_text = improver_model.improve_text(
    query=query,
    refined_text_from_website=refined_text_from_website,
    refined_text_from_corpus=refined_text_from_corpus
)

if improved_text:
    print("Improved Text generated...")
    
else:
    print("Failed to improve text.")


# Save to disk
storage_manager.save_to_folder("ImproverModelResponses", improved_text, f"{trial_number}.json")


Data loaded from ../data/WebsiteRefinerModelResponses/5.json
Data loaded from ../data/CorpusRefinerModelResponses/5.json
Improved Text generated...
Data saved to ../data/ImproverModelResponses/5.json


In [12]:
print(refined_text_from_website)

THERE IS NO RELEVANT INFORMATION IN THE RAW TEXT


In [13]:
print(improved_text['choices'][0]['message']['content'])

Life insurance is a financial product that provides coverage to policyholders in exchange for regular payments known as premiums. It is designed to protect families and provide financial security by offering a death benefit to beneficiaries upon the insured's passing. Various types of life insurance policies, such as permanent and term life insurance, cater to different needs and preferences. Permanent life insurance options, including whole life and adjustable premium policies, offer lifelong coverage with various payment structures. In contrast, term life insurance provides coverage for a specific period, such as 10, 20, or 30 years, and is typically more cost-effective for those seeking temporary protection. Life insurance aims to ensure peace of mind for families by safeguarding their financial future.


In [14]:
"""
Create multiple sets of reference material
"""

# Create a set of reference material that has the original (unimproved) content from the website

website_refiner_model_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")

unmodified_set_of_reference_material =  [
    {
        "link" : item['link'],
        "information" : item['choices'][0]['message']['content']
    }
    for item in website_refiner_model_responses
]

# Create a set of reference material that has the new (improved) content
improved_text = storage_manager.load_from_folder('ImproverModelResponses', f"{trial_number}.json")
improved_version = improved_text['choices'][0]['message']['content']


# Create a new set of reference material

new_set_of_reference_material = []

for item in unmodified_set_of_reference_material:
    if item['link'] == 'https://www.sfbli.com/':
        new_set_of_reference_material.append({"link" : link, "information" : improved_version})
    else:
        new_set_of_reference_material.append(item)



Data loaded from ../data/WebsiteRefinerModelResponses/5.json
Data loaded from ../data/ImproverModelResponses/5.json


In [ ]:
"""
Store Sets of Reference Material
"""

sets_of_reference_material = []

improvement_method = 0
set_of_reference_material = unmodified_set_of_reference_material

sets_of_reference_material.append({
    "improvement_method" : improvement_method,
    "set_of_reference_material" : set_of_reference_material,
})

improvement_method = 1
set_of_reference_material = new_set_of_reference_material

sets_of_reference_material.append({
    "improvement_method" : improvement_method,
    "set_of_reference_material" : set_of_reference_material,
})


storage_manager.save_to_folder("SetsOfReferenceMaterial", sets_of_reference_material, f"{trial_number}.json")



In [15]:

"""
Send sets of reference material to the writer-citer
"""

writer_citer_model_responses = []

# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=unmodified_set_of_reference_material
)

if cited_response:
    print("Cited response captured...")
    print(cited_response)
else:
    print("Failed to generate cited response.")
        



Cited response captured...
{'id': 'chatcmpl-AX92SnB1xEQMe3NWs5w7ry5p4SDoT', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Life insurance is a contract between a policyholder and an insurance company, where the insurer agrees to pay a specified sum of money to designated beneficiaries upon the policyholder’s death, in exchange for premium payments made by the policyholder[1][2]. The primary purpose of life insurance is to provide financial security to the insured’s dependents, helping them cover expenses such as funeral costs, debts, or living expenses[1][2].\n\nThere are various types of life insurance, including term life insurance, which provides coverage for a specific period, and whole life insurance, which offers lifelong coverage along with a savings component[1][3]. Within these categories, there are specific programs such as the Veterans Affairs Life Insurance (VALife), which is tailored for service-connected veterans, providing gua

In [16]:
cited_response['improvement_method'] = 0

writer_citer_model_responses.append(cited_response)
writer_citer_model_responses

[{'id': 'chatcmpl-AX92SnB1xEQMe3NWs5w7ry5p4SDoT',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': 'Life insurance is a contract between a policyholder and an insurance company, where the insurer agrees to pay a specified sum of money to designated beneficiaries upon the policyholder’s death, in exchange for premium payments made by the policyholder[1][2]. The primary purpose of life insurance is to provide financial security to the insured’s dependents, helping them cover expenses such as funeral costs, debts, or living expenses[1][2].\n\nThere are various types of life insurance, including term life insurance, which provides coverage for a specific period, and whole life insurance, which offers lifelong coverage along with a savings component[1][3]. Within these categories, there are specific programs such as the Veterans Affairs Life Insurance (VALife), which is tailored for service-connected veterans, providing guaranteed acce

In [17]:
# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=new_set_of_reference_material
)

if cited_response:
    print("Cited Response:")
    print(cited_response)
else:
    print("Failed to generate cited response.")

Cited Response:
{'id': 'chatcmpl-AX92bfA7F7Qe0KwgPWSFtGPpF6Odo', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': "Life insurance is a financial product designed to provide coverage and financial protection to beneficiaries in the event of the policyholder's death. It operates as a contract between an insurance company and the policyholder, whereby the insurer agrees to pay a designated sum of money to the beneficiaries upon the policyholder's death, in exchange for periodic premium payments [1][5]. \n\nLife insurance serves the primary purpose of ensuring financial security for the policyholder's loved ones, helping to cover expenses such as funerals, debts, or other financial needs that might arise after the policyholder's passing [1][5]. Additionally, some life insurance policies also offer living benefits. This provision allows a policyholder diagnosed with a covered chronic, critical, or terminal illness to access a portion of the death be

In [18]:
cited_response['improvement_method'] = 1

writer_citer_model_responses.append(cited_response)
writer_citer_model_responses

[{'id': 'chatcmpl-AX92SnB1xEQMe3NWs5w7ry5p4SDoT',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': 'Life insurance is a contract between a policyholder and an insurance company, where the insurer agrees to pay a specified sum of money to designated beneficiaries upon the policyholder’s death, in exchange for premium payments made by the policyholder[1][2]. The primary purpose of life insurance is to provide financial security to the insured’s dependents, helping them cover expenses such as funeral costs, debts, or living expenses[1][2].\n\nThere are various types of life insurance, including term life insurance, which provides coverage for a specific period, and whole life insurance, which offers lifelong coverage along with a savings component[1][3]. Within these categories, there are specific programs such as the Veterans Affairs Life Insurance (VALife), which is tailored for service-connected veterans, providing guaranteed acce

In [19]:
for item in writer_citer_model_responses:
    print(item['choices'][0]['message']['content'])
    print("---")

Life insurance is a contract between a policyholder and an insurance company, where the insurer agrees to pay a specified sum of money to designated beneficiaries upon the policyholder’s death, in exchange for premium payments made by the policyholder[1][2]. The primary purpose of life insurance is to provide financial security to the insured’s dependents, helping them cover expenses such as funeral costs, debts, or living expenses[1][2].

There are various types of life insurance, including term life insurance, which provides coverage for a specific period, and whole life insurance, which offers lifelong coverage along with a savings component[1][3]. Within these categories, there are specific programs such as the Veterans Affairs Life Insurance (VALife), which is tailored for service-connected veterans, providing guaranteed acceptance whole life coverage[2][3].

Some life insurance policies also include living benefits that allow the policyholder to access a portion of the death bene

In [20]:
# Save to disk
storage_manager.save_to_folder("WriterCiterModelResponses", writer_citer_model_responses, f"{trial_number}.json")

Data saved to ../data/WriterCiterModelResponses/5.json


In [21]:
print("Hello")

Hello


[{'improvement_method': 0,
  'set_of_reference_material': [{'link': 'https://www.libertymutual.com/insurance-resources/life/how-does-life-insurance-work',
    'information': "Life insurance is a contract between an insurance company and policyholder. In exchange for a premium, the life insurance company agrees to pay a sum of money to one or more named beneficiaries upon the death of the policyholder. The purpose of life insurance is to help provide financial security to your loved ones upon your death. Some life policies also offer living benefits, allowing payment of a part of the policy's death benefit while you're still alive if diagnosed with a covered chronic, critical, or terminal illness."},
   {'link': 'https://www.benefits.va.gov/insurance/',
    'information': "Life insurance is a financial product that provides coverage in the event of death, offering beneficiaries a payout. There are various types of life insurance available:\n\n1. **Veterans Affairs Life Insurance (VALife

Data saved to ../data/SetsOfReferenceMaterial/5.json
